In [1]:
# example of extracting and resizing faces into a new dataset
from os import listdir
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

from numpy import asarray
from numpy import savez_compressed
from PIL import Image
from mtcnn.mtcnn import MTCNN
from matplotlib import pyplot

# load an image as an rgb numpy array
def load_image(filename):
    # load image from file
    image = Image.open(filename)
    # convert to RGB, if needed
    image = image.convert('RGB')
    # convert to array
    pixels = asarray(image)
    return pixels

# extract the face from a loaded image and resize
def extract_face(model, pixels, required_size=(80, 80)):
    # detect face in the image
    faces = model.detect_faces(pixels)
    # skip cases where we could not detect a face
    if len(faces) == 0:
        return None
    # extract details of the face
    x1, y1, width, height = faces[0]['box']
    # force detected pixel values to be positive (bug fix)
    x1, y1 = abs(x1), abs(y1)
    # convert into coordinates
    x2, y2 = x1 + width, y1 + height
    # retrieve face pixels
    face_pixels = pixels[y1:y2, x1:x2]
    # resize pixels to the model size
    image = Image.fromarray(face_pixels)
    image = image.resize(required_size)
    face_array = asarray(image)
    return face_array

# load images and extract faces for all images in a directory
def load_faces(directory, n_faces):
    # prepare model
    model = MTCNN()
    faces = list()
    labels = []
    # enumerate files
    for filename in listdir(directory):
        print(filename)
        # load the image
        pixels = load_image(directory + filename)
        # get face
        face = extract_face(model, pixels)
        if face is None:
            continue
        # store
        faces.append(face)
        labels.append(filename)
        print(len(faces), face.shape)
        # stop once we have enough
        if len(faces) >= n_faces:
            break
    return asarray(faces), asarray(labels)

if __name__=='__main__':

    # directory that contains all images
    directory = '/git-repos/latent-space-arithmetic/dataset/img_align_celeba/'
    # load and extract all faces
    all_faces, face_labels = load_faces(directory, 10000)
    print('Loaded: ', all_faces.shape)
    # save in compressed format
    savez_compressed('img_align_celeba.npz', all_faces)
    savez_compressed('img_align_celeba_labels.npz', face_labels)

000001.jpg
1 (80, 80, 3)
000002.jpg
2 (80, 80, 3)
000003.jpg


KeyboardInterrupt: 

In [2]:
# load the prepared dataset
from numpy import load
# load the face dataset
data = load('img_align_celeba.npz')
faces = data['arr_0']
print('Loaded: ', faces.shape)

Loaded:  (10000, 80, 80, 3)


In [5]:
labels = load('img_align_celeba_labels.npz')['arr_0']

In [6]:
print(labels)

['000001.jpg' '000002.jpg' '000003.jpg' ... '010012.jpg' '010013.jpg'
 '010014.jpg']
